In [68]:
import minsearch

In [69]:
import json

In [70]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [71]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [72]:
index = minsearch.Index(
    text_fields = ["question", "text", "section"],
    keyword_fields = ["course"]
)

In [73]:
index.fit(documents)

In [74]:
from openai import OpenAI

In [75]:
client = OpenAI()

In [76]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query = query,
        filter_dict = {'course': 'data-engineering-zoomcamp'},
        boost_dict = boost,
        num_results = 5
    )

    return results

In [81]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. 
    Please answer the QUESTION based on the CONTEXT. 
    Please only use the facts from the CONTEXT when answering.
    If the CONTEXT doesn't contain the answer, please output NONE.
    
    QUESTION: {question}
    
    CONTEXT: {context}
    """.strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(
        question = query,
        context = context
    ).strip()

    return prompt

In [82]:
def llm(prompt):
    response = client.chat.completions.create(
        model = 'gpt-4o',
        messages = [
            {
                "role": "user",
                "content": prompt
            }
       ]
    )

    return response.choices[0].message.content

In [85]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

    return answer

In [84]:
rag('the course has already started, can I still enroll?')

"Yes, even if the course has already started, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."